# Analiza BPE tokenizera

U nastavku su opažanja primećena pri korišćenju BPE tokenizera
baziranog na pomenutom setu podataka.

In [39]:
from transformers import PreTrainedTokenizerFast

bpe_tokenizer = PreTrainedTokenizerFast(tokenizer_file="./naslovi-tokenizer.json")
simple_tokenizer = PreTrainedTokenizerFast(tokenizer_file="./naslovi-simple-tokenizer.json")
bpe_tokenizer.pad_token = "<pad>"
simple_tokenizer.pad_token = "<pad>"

def tokenizuj(tokenizer, tekst):
    print(tokenizer.tokenize(tekst), tokenizer.encode(tekst))


Prvo što primećujemo je da BPE tokenizer drugačije enkoduje reč
ako je na početku rečenice i ako je u sredini.

In [114]:
tokenizuj(bpe_tokenizer, 'Danas')
tokenizuj(bpe_tokenizer, ' Danas')

['Danas'] [1852]
['ĠDanas'] [6431]


Možemo da vidimo i da tokenizer isto tretira reči na početku
rečenice kao sufiksne nastavke koje je sam kreirao. Ovo ponašanje
je neočekivano ali se možda objašnjava postojanjem tokena za
početak rečenice u trening setu.

In [115]:
tokenizuj(bpe_tokenizer, 'Partizanove nove prinove')
tokenizuj(bpe_tokenizer, 'nove Partizanove prinove')
# obratiti pažnju na token 1923

['Partiza', 'nove', 'Ġnove', 'Ġpri', 'nove'] [1904, 1923, 1409, 399, 1923]
['nove', 'ĠPartizanove', 'Ġpri', 'nove'] [1923, 39971, 399, 1923]


Ilustrativno prikazujemo da klasičan tokenizer enkoduje ove
reči isto nezavisno od lokacije, ali na primer reč "prinove"
nema u rečniku.

In [76]:
tokenizuj(simple_tokenizer, 'Partizanove prinove')
tokenizuj(simple_tokenizer, ' Partizanove prinove')
print(simple_tokenizer.decode(simple_tokenizer.encode('Partizanove prinove')))

['Partizanove', 'prinove'] [20700, 3]
['Partizanove', 'prinove'] [20700, 3]
Partizanove <unk>


Česte reči ipak imaju svoj token u vokabularu, iako se mogu sastaviti
od drugih. Na primer, postoji "Kraljevo", ali i "Kralje" i "-vo".

In [116]:
tokenizuj(bpe_tokenizer, 'Kraljevo Kraljevo Kraljevski')
tokenizuj(bpe_tokenizer, ' Beo-grad Beograd Beogradu Beogradski')

['Kralje', 'vo', 'ĠKraljevo', 'ĠKralje', 'vski'] [15284, 287, 32176, 5095, 3841]
['ĠBeo', '-', 'grad', 'ĠBeograd', 'ĠBeogradu', 'ĠBeogradski'] [16632, 15, 3052, 1375, 986, 19432]


Iako je možda očekivano da nađemo primere gde tokenizer odvaja
osnovu reči i sufikse (na primer glagolsku osnovu i nastavke za
vreme i rod), to je slučaj samo kada reč nije dovoljno česta da
ima svoj token (npr "videvši").

In [106]:
tokenizuj(bpe_tokenizer, ' vikao vikala viče')
tokenizuj(bpe_tokenizer, ' uživao uživaju uživa')
tokenizuj(bpe_tokenizer, ' video videli videla videvši')
tokenizuj(bpe_tokenizer, ' peva pevao pevala')
tokenizuj(bpe_tokenizer, ' vozio vozi voziće vozač vozili')

['Ġvikao', 'Ġvikala', 'ĠviÄįe'] [16158, 28843, 24093]
['ĠuÅ¾ivao', 'ĠuÅ¾ivaju', 'ĠuÅ¾iva'] [25471, 11842, 4046]
['Ġvideo', 'Ġvideli', 'Ġvidela', 'Ġvide', 'vÅ¡i'] [1991, 2214, 4513, 954, 25014]
['Ġpeva', 'Ġpevao', 'Ġpevala'] [1680, 11726, 11144]
['Ġvozio', 'Ġvozi', 'Ġvozi', 'Äĩe', 'ĠvozaÄį', 'Ġvozili'] [7036, 1394, 1394, 320, 4134, 24893]


Vidimo da se složene reči koje nisu dovoljno česte da imaju
svoj token sastavljaju od drugih tokena kao što bismo očekivali
(npr. "prekosutra" iz "preko" i "-sutra")

In [117]:
tokenizuj(bpe_tokenizer, ' novonastala novogradnja i ugradnja')
tokenizuj(bpe_tokenizer, ' noć ponoć sinoć preksinoć')
tokenizuj(bpe_tokenizer, ' sutra prekosutra')
tokenizuj(bpe_tokenizer, ' puta prekoputa triputa')

['Ġnovo', 'na', 'stala', 'Ġnovo', 'gradnja', 'Ġi', 'Ġu', 'gradnja'] [1457, 263, 3829, 1457, 19600, 266, 273, 19600]
['ĠnoÄĩ', 'ĠponoÄĩ', 'ĠsinoÄĩ', 'Ġpre', 'ksi', 'noÄĩ'] [2902, 15056, 6288, 351, 1165, 5087]
['Ġsutra', 'Ġpreko', 'sutra'] [1648, 1459, 31974]
['Ġputa', 'Ġpreko', 'puta', 'Ġtri', 'puta'] [1474, 1459, 8308, 917, 8308]
